# TODO 
    1) Move Credentials into config - Done
    2) Break out all the modules - Done
    3) Configure for selenium
    4) Deploy to pi

In [41]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from time import sleep
import json

In [42]:
import get_params as gp
import chk_new_res as cn
import build_payload as bp
import send2cal as cal
from importlib import reload
reload(gp)

<module 'get_params' from '/mnt/c/Users/brene/OneDrive - Grand Canyon University/DATASCIENCE/Documentation/Documentation_5_25_20/TigerTeamAnalytics/TigerProjects/Scrapers/GrandWelcome/Scraper/app/get_params.py'>

In [43]:
cred = gp.get_params('login')
# for jupyter notebook
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# For selenium
# browser = webdriver.Remote('http://selenium:4444/wd/hub', desired_capabilities=DesiredCapabilities.CHROME)


In [44]:
browser.get(str(cred['url']))

In [45]:

# Login 
username=browser.find_element(by=By.XPATH, value="//input[@placeholder='Email']")
password=browser.find_element(by=By.XPATH, value="//input[@placeholder='Password']")
login=browser.find_element(by=By.XPATH, value="//button[@type='submit']")
username.send_keys(str(cred['user']))
password.send_keys(str(cred['password']))
login.submit()

In [46]:
# # get reservations 
restab=browser.find_element(by=By.XPATH, value="//span[normalize-space()='Reservations']")
restab.click()
sleep(5)

In [51]:
from selenium.webdriver.support.ui import Select
# browser.find_element(by=By.XPATH, value="//*[@id='table_length']/label/select")

<selenium.webdriver.remote.webelement.WebElement (session="69d52f02b57377235d1eccc599ea3c01", element="807690da-6472-4aa5-8e8d-758acd97fd10")>

In [52]:
select = Select(browser.find_element(by=By.XPATH, value="//*[@id='table_length']/label/select"))
# select by value 
select.select_by_value('100')

In [53]:
# check if another page 
table=browser.find_element(by=By.XPATH, value='//*[@id="table_paginate"]/ul')
items = table.text.split()
items.remove('Previous')
items.remove('Next')

In [54]:
#### The html table is the same table. ignore this and use html web
# export=browser.find_element(by=By.XPATH, value="//input[@value='Export']")
# export.click()

# multiple pages
webtables = []
for page in items:
    # set to 100 
    select = Select(browser.find_element(by=By.XPATH, value="//*[@id='table_length']/label/select"))
    # select by value 
    select.select_by_value('100')
    # get reservations from web
    html = browser.page_source
    table = pd.read_html(html)
    res_raw_df = table[0]
    webtables.append(res_raw_df)
    # click next page 
    nxt=browser.find_element(by=By.XPATH, value='//*[@id="table_next"]/a')
    nxt.click()  
    

In [55]:
combined=pd.concat(webtables)
res_df=combined[['Res. #', 'Status', 'Unit', 'Guest', 'Booked Date', 'Check-In','Checkout', 'Nights', 'Income']]

In [570]:
# merge with old data
rpts=gp.get_params('reports')
old=pd.read_csv(rpts['old'])
frames = [res_df,old]
summ=pd.concat(frames)

In [572]:
# create a summary table 
summ1=summ[['Check-In','Checkout', 'Nights', 'Income']]
nightly_rate = []
for i,b in summ1.iterrows():
    Arrival = b['Check-In']
    Departure = b['Checkout']
    Income_raw = b['Income']
    Income= float(sub(r'[^\d.]', '', Income_raw))
    times=CountDaysMonth(Arrival,Departure)
    for t in times:
        nightly = Income / len(times)
        night_form = '{0:.2f}'.format(nightly)
        t_stmp=pd.Timestamp(t)
        nightly_rate.append([t_stmp,float(night_form)])
breakdown = pd.DataFrame(nightly_rate, columns = ['Date', 'Per Night'])
breakdown['Month-Year']=breakdown['Date'].dt.strftime('%m/%Y')
rpts=gp.get_params('reports')
breakdown.to_csv(rpts['breakdown'],index=False)

In [573]:
# create current month 
current_month=datetime.now().strftime('%m/%Y')
month_filter=breakdown['Month-Year'] == current_month
curr=breakdown[month_filter]

In [580]:
breakdown

,Date,Per Night,Month-Year
0,2022-09-23,0.00,2022-09-01
1,2022-09-24,0.00,2022-09-01
2,2022-09-25,0.00,2022-09-01
3,2022-09-26,0.00,2022-09-01
4,2022-09-27,0.00,2022-09-01
...,...,...,...
333,2022-07-22,211.47,2022-07-01
334,2022-07-23,211.47,2022-07-01
335,2022-07-24,211.47,2022-07-01
336,2022-07-25,211.47,2022-07-01


In [577]:
df['date_col'] =  pd.to_datetime(df['date_col'], format='%Y/%m/%d')

In [579]:

# breakdown['Month-Year'] =  pd.to_datetime(breakdown['Month-Year'], format='%m/%Y')
year_filter

,Date,Per Night,Month-Year
0,2022-09-23,0.00,2022-09-01
1,2022-09-24,0.00,2022-09-01
2,2022-09-25,0.00,2022-09-01
3,2022-09-26,0.00,2022-09-01
4,2022-09-27,0.00,2022-09-01
...,...,...,...
333,2022-07-22,211.47,2022-07-01
334,2022-07-23,211.47,2022-07-01
335,2022-07-24,211.47,2022-07-01
336,2022-07-25,211.47,2022-07-01


In [217]:
# pivot chart
df=breakdown.groupby([breakdown['Date'].dt.year, breakdown['Date'].dt.month])['Per Night'].sum().to_frame()
df.index.names = ['Year','Month']
costs = gp.get_params('expenses')
total_cost = float(costs['mortgage'])+float(costs['club'])+float(costs['resort'])+float(costs['avg_operating'])
df["Profit/Loss"] = df["Per Night"] - total_cost
df

,Check-In,Checkout,Nights,Income
0,"Sep 23, 2022","Oct 2, 2022",9,$0.00
1,"Oct 14, 2022","Oct 19, 2022",5,$562.27
2,"Oct 6, 2022","Oct 12, 2022",6,$674.73
3,"Nov 9, 2022","Nov 13, 2022",4,$468.18
4,"Oct 22, 2022","Oct 30, 2022",8,$899.64
5,"Dec 23, 2022","Jan 2, 2023",10,"$2,582.61"
6,"Nov 23, 2022","Nov 26, 2022",3,$506.43
7,"Nov 19, 2022","Nov 23, 2022",4,$421.51
8,"Nov 6, 2022","Nov 9, 2022",3,$314.41
9,"Dec 3, 2022","Dec 10, 2022",7,$733.63


# Chart summary 

In [ ]:
# Stream Lit requires that the py file be run directly. 
# so costs and breakdown will have to be build here. 

In [254]:
import streamlit as st
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
# Import breakdown
rpts=gp.get_params('reports')
costs = gp.get_params('expenses')
breakdown = pd.read_csv(rpts['breakdown'])

# Build chart
fig = px.bar(breakdown,x='Month-Year',  y='Per Night', barmode='group')

fig.add_hrect(y0=0, y1=costs['mortgage'], line_width=0, fillcolor="red", opacity=0.25)
fig.add_hrect(y0=costs['mortgage'], y1=costs['mortgage']+costs['avg_operating'], line_width=0, fillcolor="yellow", opacity=0.25)
fig.add_hrect(y0=costs['mortgage']+costs['avg_operating'], y1=10000, line_width=0, fillcolor="green", opacity=0.25)

# fig.show()
st.plotly_chart(fig, use_container_width=True)

In [613]:
costs = gp.get_params('expenses')
Mrt_HOA=float(costs['mortgage'])+float(costs['club'])+float(costs['resort'])
Mrt_HOA
op_cost=float(costs['avg_operating'])

In [614]:
breakdown['MortgageHoa'] = Mrt_HOA
breakdown['opCost'] = op_cost
breakdown

,Date,Per Night,Month-Year,MortgageHoa,opCost
0,2022-09-23,0.00,2022-09-01,2132.81,1000.0
1,2022-09-24,0.00,2022-09-01,2132.81,1000.0
2,2022-09-25,0.00,2022-09-01,2132.81,1000.0
3,2022-09-26,0.00,2022-09-01,2132.81,1000.0
4,2022-09-27,0.00,2022-09-01,2132.81,1000.0
...,...,...,...,...,...
333,2022-07-22,211.47,2022-07-01,2132.81,1000.0
334,2022-07-23,211.47,2022-07-01,2132.81,1000.0
335,2022-07-24,211.47,2022-07-01,2132.81,1000.0
336,2022-07-25,211.47,2022-07-01,2132.81,1000.0


In [587]:
# find number of months ( 1st day of months aka mortgage day) between two dates. 
# get all the dates 


DTSTART:20210301T000000
RRULE:FREQ=MONTHLY;UNTIL=20221108T000000;BYMONTHDAY=1


# OLD DATA

In [555]:
import os 
# os.listdir('old_data/RentalActivity')

In [556]:
# old_rr = pd.read_csv('old_data/RentalActivity/rentalactivity_April-2021.csv')

In [334]:
parsed

[['29725722', '3/28/2021', '4/4/2021', 1352.83],
 ['30353982', '4/4/2021', '4/10/2021', 1225.31],
 ['30573076', '4/10/2021', '4/17/2021', 1394.65],
 ['31704950', '7/30/2021', '8/7/2021', 1376.99],
 ['30247918', '6/26/2021', '7/26/2021', 3652.86],
 ['30905584', '5/6/2021', '6/8/2021', 4308.81],
 ['31677686', '6/9/2021', '6/16/2021', 1160.0],
 ['30502479', '6/17/2021', '6/23/2021', 1099.35],
 ['29764605', '3/22/2021', '3/28/2021', 1147.5],
 ['30627570', '4/17/2021', '5/4/2021', 3195.46],
 ['35512077', '11/20/2021', '11/25/2021', 840.2]]

In [595]:
alltimes=CountDaysMonth('03/01/2021',datetime.now().strftime('%m/%d/%Y'))

In [592]:
datetime.now().strftime('%m/%d/%Y')
from dateutil.parser import parse

'11/08/2022'

In [608]:
import locale 
locale.setlocale(locale.LC_ALL, 'en_US.utf8')


'en_US.utf8'

In [610]:
example =51744.47
print(locale.currency(example, grouping=True))

$51,744.47


In [110]:
for t in times:
    nightly = Income / len(times)
    night_form = '{0:.2f}'.format(nightly)
    t_stmp=pd.Timestamp(t)
    nightly_rate.append([t_stmp,Decimal(night_form)]) 

In [112]:
breakdown = pd.DataFrame(nightly_rate, columns = ['Date', 'Per Night'])

In [ ]:
# need to build full dataframe before we groupy by


In [145]:
df=breakdown.groupby([breakdown['Date'].dt.year, breakdown['Date'].dt.month])['Per Night'].sum().to_frame()
df.index.names = ['Year','Month']

In [146]:
df

,,Per Night
Year,Month,
2022,12,2324.34
2023,1,258.26


In [594]:
    def CountDaysMonth(Arrival,Departure):    
        from datetime import date, timedelta
        from dateutil.parser import parse
        from datetime import datetime
        BookedDaysPerMonth = {}
        d1 = parse(Arrival).date()
        d2 = parse(Departure).date()
        # this will give you a list containing all of the dates
        times = [d1 + timedelta(days=x) for x in range((d2-d1).days + 1)]
        # Creates a dictionary and increments if it exists 
        for t in times:
            month = t.strftime("%b")+'_'+t.strftime("%Y")
            if month in BookedDaysPerMonth:
                BookedDaysPerMonth[month] += 1
            else:
                BookedDaysPerMonth[month] = 1
        Dates_list=[]
        for k,v in BookedDaysPerMonth.items():
            Dates_list.append('{}-{}'.format(k,v))
        return Dates_list

In [522]:
res_file=gp.get_params('reservations')
# check if res id exists
curr = pd.read_csv(res_file['all_res'])
new_res=res_df[['Res. #','Guest','Check-In','Checkout','Nights']]
new_res.columns = ['Res_ID','Guest','CheckIn','CheckOut','Nights']
n_df=cn.checkNewRes(curr,new_res)

In [75]:
# Send new to calendar
result = n_df.to_json(orient="records")
n_pl=json.loads(result)
for rec in n_pl:
    cal.Send2Cal(bp.build_pl(rec))

4806brierrose@brenesfinancial.com
Event created: https://www.google.com/calendar/event?eid=NGlzajZmMHBhMGhzaW11dmkwNnJ0OGFwZmMgNDgwNmJyaWVycm9zZUBicmVuZXNmaW5hbmNpYWwuY29t


In [ ]:
# save updated file 
update =[n_df,curr]
master_df = pd.concat(update)
master_df.reset_index(drop=True,inplace=True)
master_df.to_csv(res_file['all_res'],index=False)

# Merge OLD data for one-time final 

In [429]:
import os 

parsed2 = []
for file in os.listdir('old_data'):
    old_dl = pd.read_excel('old_data/'+file)
    
    # get the right table and delete noise
    t1=old_dl[10:].T
    crap=t1[10].notnull()
    t2=t1[crap].T
    col=['Date','Description','Category','Credit','Debit']
    t2.columns = col
    t3=t2.dropna(axis=0, how='all')
    newcols=['Date','Description','Category','Amount']
    rent_filter= t3['Category'] == 'Rental Income'
    t4=t3[rent_filter]
    t5=t4[['Date','Description','Category','Credit']]
    t5.columns = newcols

for i,rec in t5.iterrows():
    print("inside final loop")
    try:
        cleaner=rec['Description'].split(' ')[0]+' '+rec['Description'].split(' ')[1]
        if cleaner == 'Rental payment':
            ID=rec['Description'].split('BID:')[1].split(')')[0].strip()
            arrival=rec['Description'].split(')')[1].split('->')[0].strip()
            departure = rec['Description'].split(')')[1].split('->')[1].strip()
            Income=rec['Amount']
            parsed2.append([ID,arrival, departure,Income])
        
    except:
        print("Lost it in the final scrub")

10_2021_Monthly_Report_Download.xls
11_2021_Monthly_Report_Download.xls
12_2021_Monthly_Report_Download.xls
1_2021_Monthly_Report_Download.xls
1_2022_Monthly_Report_Download.xls
2_2021_Monthly_Report_Download.xls
2_2022_Monthly_Report_Download.xls
3_2022_Monthly_Report_Download.xls
4_2021_Monthly_Report_Download.xls
4_2022_Monthly_Report_Download.xls
5_2022_Monthly_Report_Download.xls
6_2021_Monthly_Report_Download.xls
6_2022_Monthly_Report_Download.xls
7_2021_Monthly_Report_Download.xls
7_2022_Monthly_Report_Download.xls
8_2021_Monthly_Report_Download.xls
9_2021_Monthly_Report_Download.xls


In [372]:
t1=old_dl[10:].T
crap=t1[10].notnull()
t2=t1[crap].T
col=['Date','Description','Category','Credit','Debit']
t2.columns = col
t3=t2.dropna(axis=0, how='all')

In [381]:
newcols=['Date','Description','Category','Amount']
rent_filter= t3['Category'] == 'Rental Income'
t4=t3[rent_filter]
t5=t4[['Date','Description','Category','Credit']]
t5.columns = newcols


,Date,Description,Category,Amount
10,Date,Description,Rental Income,Credit
13,2022-06-01 00:00:00,Bill: Kissimmee Utility Authority / Service Pe...,Rental Income,NaN
16,2022-06-01 00:00:00,Monthly Pest control,Rental Income,NaN
18,2022-06-01 00:00:00,Weekly Pool Care,Rental Income,NaN
20,2022-06-01 00:00:00,Monthly Management fee,Rental Income,NaN
22,2022-06-01 00:00:00,Clean for Mike Brenes (01 June 22),Rental Income,NaN
24,2022-06-01 00:00:00,Bill: Toho Water Authority / Service Period 04...,Rental Income,NaN
27,2022-06-01 00:00:00,Bill payment - service period 04/20/22 - 06/19/22,Rental Income,NaN
29,2022-06-05 00:00:00,Replace A/C filter,Rental Income,NaN
31,2022-06-08 00:00:00,Rental payment for Peter Quinones (BID: 362782...,Rental Income,1209.6


In [409]:
parsed2 = []
for i,rec in t5.iterrows():
    try:
        cleaner=rec['Description'].split(' ')[0]+' '+rec['Description'].split(' ')[1]
        if cleaner == 'Rental payment':
            ID=rec['Description'].split('BID:')[1].split(')')[0].strip()
            arrival=rec['Description'].split(')')[1].split('->')[0].strip()
            departure = rec['Description'].split(')')[1].split('->')[1].strip()
            Income=rec['Amount']
            parsed2.append([ID,arrival, departure,Income])
        
    except:
        None

In [410]:
parsed2

[['36278294', '6/1/2022', '6/8/2022', 1209.6],
 ['36204474', '6/13/2022', '6/19/2022', 1086.65]]

In [378]:
parsed= []
for i,rec in t5.iterrows():
    cleaner=rec['Description'].split(' ')[0]
    if cleaner == 'Rental payment':
        ID=rec['Description'].split('BID:')[1].split(')')[0].strip()
        arrival=rec['Description'].split(')')[1].split('-')[0].strip()
        departure = rec['Description'].split(')')[1].split('-')[1].strip()
        Income=rec['Amount']
        parsed.append([ID,arrival, departure,Income])

In [436]:
my_df = []
for file in os.listdir('old_data'):
    old_dl = pd.read_excel('old_data/'+file)
    print(file)
    # get the right table and delete noise
    t1=old_dl[10:].T
    crap=t1[10].notnull()
    t2=t1[crap].T
    col=['Date','Description','Category','Credit','Debit']
    t2.columns = col
    t3=t2.dropna(axis=0, how='all')
    my_df.append(t3)   


10_2021_Monthly_Report_Download.xls
11_2021_Monthly_Report_Download.xls
12_2021_Monthly_Report_Download.xls
1_2021_Monthly_Report_Download.xls
1_2022_Monthly_Report_Download.xls
2_2021_Monthly_Report_Download.xls
2_2022_Monthly_Report_Download.xls
3_2022_Monthly_Report_Download.xls
4_2021_Monthly_Report_Download.xls
4_2022_Monthly_Report_Download.xls
5_2022_Monthly_Report_Download.xls
6_2021_Monthly_Report_Download.xls
6_2022_Monthly_Report_Download.xls
7_2021_Monthly_Report_Download.xls
7_2022_Monthly_Report_Download.xls
8_2021_Monthly_Report_Download.xls
9_2021_Monthly_Report_Download.xls


In [ ]:
nothing = [0,3,7,16]
something = [1,2,4,6,8,9,10,11,12,13,14,15]

In [511]:
check=my_df[17]
newcols=['Date','Description','Category','Amount']

IndexError: list index out of range

In [508]:
rent_filter= check['Category'] == 'Rental Income'
t4=check[rent_filter]
check[rent_filter]

,Date,Description,Category,Credit,Debit


In [509]:
t5=t4[['Date','Description','Category','Credit']]
t5.columns = newcols
t5

,Date,Description,Category,Amount


In [510]:
for i,rec in t5.iterrows():
    cleaner=rec['Description'].split(' ')[0]
    if cleaner == 'Rental':
        ID=rec['Description'].split('BID:')[1].split(')')[0].strip()
        arrival=rec['Description'].split(')')[1].split('->')[0].strip()
        departure = rec['Description'].split(')')[1].split('->')[1].strip()
        Income=rec['Amount']
        parsed2.append([ID,arrival, departure,Income])

In [515]:
parsed2 = []
something = [1,2,4,6,8,9,10,11,12,13,14,15]
newcols=['Date','Description','Category','Amount']

In [518]:
for df in something:
    check=my_df[df]
    rent_filter= check['Category'] == 'Rental Income'
    t4=check[rent_filter]
    t5=t4[['Date','Description','Category','Credit']]
    t5.columns = newcols
    
    for i,rec in t5.iterrows():
        cleaner=rec['Description'].split(' ')[0]
        if cleaner == 'Rental':
            ID=rec['Description'].split('BID:')[1].split(')')[0].strip()
            arrival=rec['Description'].split(')')[1].split('->')[0].strip()
            departure = rec['Description'].split(')')[1].split('->')[1].strip()
            Income=rec['Amount']
            parsed2.append([ID,arrival, departure,Income])

In [519]:
parsed2

[['35512077', '11/20/2021', '11/25/2021', 840.2],
 ['35587706', '12/20/2021', '12/24/2021', 654.92],
 ['30054645', '12/26/2021', '1/2/2022', 1228.74],
 ['35383346', '2/2/2022', '2/28/2022', 4845.83],
 ['29725722', '3/28/2021', '4/4/2021', 1352.83],
 ['30353982', '4/4/2021', '4/10/2021', 1225.31],
 ['30573076', '4/10/2021', '4/17/2021', 1394.65],
 ['35437583', '3/5/2022', '4/3/2022', 3929.19],
 ['36131706', '4/5/2022', '4/8/2022', 744.2],
 ['35901766', '4/13/2022', '4/17/2022', 621.05],
 ['36051771', '4/20/2022', '4/25/2022', 897.5],
 ['36292264', '4/27/2022', '5/27/2022', 3923.9],
 ['30905584', '5/6/2021', '6/8/2021', 4308.81],
 ['31677686', '6/9/2021', '6/16/2021', 1160],
 ['30502479', '6/17/2021', '6/23/2021', 1099.35],
 ['36278294', '6/1/2022', '6/8/2022', 1209.6],
 ['36204474', '6/13/2022', '6/19/2022', 1086.65],
 ['30247918', '6/26/2021', '7/26/2021', 3652.86],
 ['35761071', '6/24/2022', '7/5/2022', 1847.81],
 ['36066025', '7/7/2022', '7/16/2022', 1568.98],
 ['36667491', '7/21/202

In [527]:
# curr
missing=pd.read_csv('reports/oldFinal_Missing.csv')

In [557]:
missing['Income'] = missing['Income'].fillna(0)
# missing

## This is the loop that was used to fill gaps in income 0

In [548]:
mylist = []
for i,r in missing.iterrows():
    resnum=str(r['Res. #'])
    status=str(r['Status'])
    unit=str(r['Unit'])
    guest=str(r['Guest'])
    booked=str(r['Booked Date'])
    chkin=str(r['Check-In'])
    chkout=str(r['Checkout'])
    nights=str(r['Nights'])
    if r['Income'] == 0:
        for i in parsed2:
            if resnum in i:
                print(resnum)
                income = i[3]
                myval = [resnum,status,unit,guest,booked,chkin,chkout,nights, '$'+str(income)]
    else:
        myval = [resnum,status,unit,guest,booked,chkin,chkout,nights, str(r['Income'])]
    mylist.append(myval) 
    


35587706
30054645
35383346
35437583
36131706
35901766
36051771
36292264
36278294
36204474
35761071
36066025
36667491


In [550]:
# safe as data frame, remove dupes, and send to csv file
mycols=['Res. #', 'Status', 'Unit', 'Guest', 'Booked Date', 'Check-In','Checkout', 'Nights', 'Income']
Final_old2 = pd.DataFrame(mylist,columns=mycols)
len(Final_old2)

25

In [552]:
Final_old2=Final_old2.drop_duplicates(keep='first', inplace=False)
len(Final_old2)
Final_old2.to_csv('reports/oldFinal.csv',index=False)

25

In [558]:
Final_old2

,Res. #,Status,Unit,Guest,Booked Date,Check-In,Checkout,Nights,Income
0,29764605,Checked Out,Legendary Times at Story Lake,Katie Dwyer,Mar/01/2021,Mar/22/2021,Mar/28/2021,6,$1147.5
1,29725722,Checked Out,Legendary Times at Story Lake,April Mial,Mar/01/2021,Mar/28/2021,Apr/04/2021,7,$1352.83
2,30353982,Checked Out,Legendary Times at Story Lake,Asil Abuhadba,Mar/01/2021,Apr/04/2021,Apr/10/2021,6,$1225.31
3,30573076,Checked Out,Legendary Times at Story Lake,Brett Barnett,Mar/01/2021,Apr/10/2021,Apr/17/2021,7,$1394.65
4,30627570,Checked Out,Legendary Times at Story Lake,Ricardo Flores,Mar/01/2021,Apr/17/2021,May/04/2021,17,$3195.46
5,30905584,Checked Out,Legendary Times at Story Lake,Martin Olano,Mar/01/2021,May/06/2021,Jun/08/2021,33,$4308.81
6,31677686,Checked Out,Legendary Times at Story Lake,Daniel Lopez,Mar/01/2021,Jun/09/2021,Jun/16/2021,7,$1160.0
7,30502479,Checked Out,Legendary Times at Story Lake,April Weldon,Mar/01/2021,Jun/17/2021,Jun/23/2021,6,$1099.35
8,30247918,Checked Out,Legendary Times at Story Lake,Jorge Elizondo,Mar/01/2021,Jun/26/2021,Jul/26/2021,30,$3652.86
9,31704950,Checked Out,Legendary Times at Story Lake,Robert Villalta,Mar/01/2021,Jul/30/2021,Aug/07/2021,8,$1376.99


In [559]:
curr

,Res_ID,Guest,CheckIn,CheckOut,Nights
0,1102691,Moore,"Dec 3, 2022","Dec 10, 2022",7
1,1102644,Figueroa,"Nov 6, 2022","Nov 9, 2022",3
2,1102386,Philbrook,"Nov 19, 2022","Nov 23, 2022",4
3,1100775,Russell,"Nov 23, 2022","Nov 26, 2022",3
4,1099938,rippy,"Dec 23, 2022","Jan 2, 2023",10
5,1098795,Reed,"Oct 22, 2022","Oct 30, 2022",8
6,1098288,Hool,"Nov 9, 2022","Nov 13, 2022",4
7,1097362,Cutcher,"Oct 6, 2022","Oct 12, 2022",6
8,1096930,Sarousi,"Oct 14, 2022","Oct 19, 2022",5
9,1096275,Brenes,"Sep 23, 2022","Oct 2, 2022",9
